# Install

In [ ]:
! git clone https://github.com/OpenNMT/OpenNMT-py.git
! pip install -r OpenNMT-py/requirements.txt

# Quickstart
動作確認
http://opennmt.net/OpenNMT-py/quickstart.html

In [10]:
base_dir = 'OpenNMT-py/data/{}'
src_train_path = base_dir.format('src-train.txt')
tgt_train_path = base_dir.format('tgt-train.txt')
src_val_path = base_dir.format('src-val.txt')
tgt_val_path = base_dir.format('tgt-val.txt')
save_data = base_dir.format('demo')

### Step 1: Preprocess the data

In [18]:
preprocess_command = f'python OpenNMT-py/preprocess.py -train_src {src_train_path} -train_tgt {tgt_train_path} -valid_src {src_val_path} -valid_tgt {tgt_val_path} -save_data {save_data}'

In [19]:
preprocess_command

'python OpenNMT-py/preprocess.py -train_src OpenNMT-py/data/src-train.txt -train_tgt OpenNMT-py/data/tgt-train.txt -valid_src OpenNMT-py/data/src-val.txt -valid_tgt OpenNMT-py/data/tgt-val.txt -save_data OpenNMT-py/data/demo'

In [22]:
! $preprocess_command

[2019-05-27 12:35:48,495 INFO] Extracting features...
[2019-05-27 12:35:48,495 INFO]  * number of source features: 0.
[2019-05-27 12:35:48,495 INFO]  * number of target features: 0.
[2019-05-27 12:35:48,495 INFO] Building `Fields` object...
[2019-05-27 12:35:48,495 INFO] Building & saving training data...
[2019-05-27 12:35:48,495 INFO] Reading source and target files: OpenNMT-py/data/src-train.txt OpenNMT-py/data/tgt-train.txt.
[2019-05-27 12:35:48,501 INFO] Building shard 0.
[2019-05-27 12:35:48,966 INFO]  * saving 0th train data shard to OpenNMT-py/data/demo.train.0.pt.
[2019-05-27 12:35:49,628 INFO]  * tgt vocab size: 35820.
[2019-05-27 12:35:49,683 INFO]  * src vocab size: 24997.
[2019-05-27 12:35:49,972 INFO] Building & saving validation data...
[2019-05-27 12:35:49,972 INFO] Reading source and target files: OpenNMT-py/data/src-val.txt OpenNMT-py/data/tgt-val.txt.
[2019-05-27 12:35:49,973 INFO] Building shard 0.
[2019-05-27 12:35:50,042 INFO]  * saving 0th valid data shard to Open

### Step 2: Train the model

速度比較メモ。GPU入れると27倍早い。


| spec                                    | tok/s            |
| --------------------------------------- | ---------------- |
| n1-standard-8（vCPU x 8、メモリ 30 GB） | 576/570：×1      |
| + GPU（Tesla V100-SXM2-16GB）           | 15566/15729：×27 |



In [23]:
save_model = 'demo-model'

In [24]:
gpu_option = '-world_size 1 -gpu_ranks 0'

In [25]:
tensorboard_log_dir = './logs/demo'
tensorboard_option = f'--tensorboard --tensorboard_log_dir {tensorboard_log_dir}'
! mkdir $tensorboard_log_dir

mkdir: cannot create directory ‘./logs/demo’: File exists


In [28]:
train_command = f'python OpenNMT-py/train.py -data {save_data} -save_model {save_model} {gpu_option} {tensorboard_option}'
! echo $train_command
! $train_command

python OpenNMT-py/train.py -data OpenNMT-py/data/demo -save_model demo-model -world_size 1 -gpu_ranks 0 --tensorboard --tensorboard_log_dir ./logs/demo
[2019-05-27 12:37:21,185 INFO]  * src vocab size = 24997
[2019-05-27 12:37:21,185 INFO]  * tgt vocab size = 35820
[2019-05-27 12:37:21,185 INFO] Building model...
[2019-05-27 12:37:27,119 INFO] NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(24997, 500, padding_idx=1)
        )
      )
    )
    (rnn): LSTM(500, 500, num_layers=2, dropout=0.3)
  )
  (decoder): InputFeedRNNDecoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(35820, 500, padding_idx=1)
        )
      )
    )
    (dropout): Dropout(p=0.3)
    (rnn): StackedLSTM(
      (dropout): Dropout(p=0.3)
      (layers): ModuleList(
        (0): LSTMCell(1000, 500)
        (1): LSTMCell(500, 500)